<a href="https://colab.research.google.com/github/parreirahpo/FormacaoPython_DIO/blob/main/Azure_DevOps_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalando as dependências necessárias**

In [ ]:
!pip install azure-devops
!pip install requests

**Realizando a autenticação com o Azure DevOps através de Personal Access Token**

In [9]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
from google.colab import userdata
import os

organization_url = f'https://dev.azure.com/{userdata.get("azureOrg")}'
project_name = userdata.get("azureProject")
personal_access_token = userdata.get("azurePat")

credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)
wit_client = connection.clients.get_work_item_tracking_client()


**Analisa o AzureDevOps em busca de todos os tipos de Work Items e seus respectivos campos, e então armazena essas informações em um Excel.**

In [ ]:
import requests
import base64
import pandas as pd

def get_field_details(url, personal_access_token):
    encoded_pat = base64.b64encode((":" + personal_access_token).encode("ascii")).decode("ascii")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Authorization': 'Basic ' + encoded_pat,
        'Accept': 'application/json'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Obtém todos os tipos de work items do projeto #
work_item_types = wit_client.get_work_item_types(project_name)

# Dicionário para armazenar os dataframes de cada tipo de work item #
work_item_fields_dfs = {}

for work_item_type in work_item_types:
    fields = work_item_type.fields
    fields_data = []

    # Itera sobre cada field e faz uma requisição para obter os detalhes #
    for field in fields:
        try:
            field_details = get_field_details(field.url, personal_access_token)
            field_info = {
                "Name": field_details.get('name', 'None'),
                "Reference Name": field_details.get('referenceName', 'None'),
                "Type": field_details.get('type', 'None'),
                "Description": field_details.get('description', '')
            }
            fields_data.append(field_info)
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred for work item type {work_item_type.name} and field {field.name}: {http_err} - URL: {field.url}")
        except Exception as e:
            print(f"An error occurred for work item type {work_item_type.name} and field {field.name}: {e} - URL: {field.url}")

    # Cria um dataframe a partir dos dados coletados #
    fields_df = pd.DataFrame(fields_data, columns=["Name", "Reference Name", "Type", "Description"])

    # Armazena o dataframe no dicionário com o nome do tipo de work item como chave #
    work_item_fields_dfs[work_item_type.name] = fields_df

# Salva todos os dataframes em um único arquivo excel com diferentes abas #
output_filename = 'work_item_types.xlsx'
with pd.ExcelWriter(output_filename) as writer:
    for work_item_type, df in work_item_fields_dfs.items():
        df.to_excel(writer, sheet_name=work_item_type, index=False)


**Através de uma query criada no Azure DevOps, busca as informações de cada Work Item retornado.**

In [13]:
# Obtém os work items através de um query_id do Azure DevOps #
query_id = userdata.get("azureQueryId")
query_results = wit_client.query_by_id(query_id)
work_items = query_results.work_items

# Dicionário para armazenar os dataframes dos estados atuais de cada tipo de work item #
work_item_current_state_dfs = {}

# Processa cada work item retornado pela query #
for work_item_ref in work_items:
    work_item_id = work_item_ref.id
    work_item_details = wit_client.get_work_item(work_item_id, expand='All')
    work_item_type = work_item_details.fields['System.WorkItemType']

    # Obtém o dataframe dos fields para o tipo de work item #
    fields_df = work_item_fields_dfs.get(work_item_type)
    if fields_df is None:
        continue

    # Dicionário para armazenar os dados do estado atual do work item #
    current_state_data = {
        'Type': work_item_type,
        'ID': work_item_id
    }

    # Preenche o dicionário com os dados atuais do work item #
    for _, field_info in fields_df.iterrows():
        field_name = field_info['Reference Name']
        field_value = work_item_details.fields.get(field_name, 'None')

        # Verifica se o valor é um dicionário e contém 'displayName' para capturar apenas o nome do usuário #
        if isinstance(field_value, dict) and 'displayName' in field_value:
            field_value = field_value['displayName']
        current_state_data[field_info['Name']] = field_value

    # Cria um dataframe a partir dos dados do estado atual #
    current_state_df = pd.DataFrame([current_state_data])

    # Armazena o dataframe no dicionário com o nome do tipo de work item como chave #
    if work_item_type not in work_item_current_state_dfs:
        work_item_current_state_dfs[work_item_type] = []
    work_item_current_state_dfs[work_item_type].append(current_state_df)

# Salva todos os dataframes de estados atuais em um único arquivo excel #
output_filename = 'work_item_current_states.xlsx'
with pd.ExcelWriter(output_filename) as writer:
    for work_item_type, dfs in work_item_current_state_dfs.items():
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_excel(writer, sheet_name=work_item_type, index=False)


**Através de uma query criada no Azure DevOps, busca pelas modificações que ocorreram em cada um dos tipos de Work Item retornados.**

In [11]:
# Obtém os work items através de um query_id do Azure DevOps #
query_id = userdata.get("azureQueryId")
query_results = wit_client.query_by_id(query_id)
work_items = query_results.work_items

# Dicionário para armazenar os dataframes de revisões de cada tipo de work item #
work_item_revisions_dfs = {}

# Processa cada work item retornado pela query #
for work_item_ref in work_items:
    work_item_id = work_item_ref.id
    work_item_details = wit_client.get_work_item(work_item_id, expand='All')
    work_item_type = work_item_details.fields['System.WorkItemType']

    # Obtém o dataframe dos fields para o tipo de work item #
    fields_df = work_item_fields_dfs.get(work_item_type)
    if fields_df is None:
        continue

    # Obtém o histórico de revisões para o work item #
    revisions = wit_client.get_revisions(work_item_id)

    # Lista para armazenar os dados das revisões #
    revisions_data = []

    for revision in revisions:
        revision_data = {
            'Type': work_item_type,
            'ID': work_item_id,
            'REV ID': revision.rev
        }
        for _, field_info in fields_df.iterrows():
            field_name = field_info['Reference Name']
            field_value = revision.fields.get(field_name, work_item_details.fields.get(field_name, 'None'))
            # Verifica se o valor é um dicionário e contém 'displayName' para capturar apenas o nome do usuário #
            if isinstance(field_value, dict) and 'displayName' in field_value:
                field_value = field_value['displayName']
            revision_data[field_info['Name']] = field_value
        revisions_data.append(revision_data)

    # Cria um dataframe a partir dos dados das revisões #
    revisions_df = pd.DataFrame(revisions_data)

    # Armazena o dataframe no dicionário com o nome do tipo de work item como chave #
    if work_item_type not in work_item_revisions_dfs:
        work_item_revisions_dfs[work_item_type] = []
    work_item_revisions_dfs[work_item_type].append(revisions_df)

# Salva todos os dataframes de revisões em um único arquivo excel #
output_filename = 'work_item_revisions.xlsx'
with pd.ExcelWriter(output_filename) as writer:
    for work_item_type, dfs in work_item_revisions_dfs.items():
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_excel(writer, sheet_name=work_item_type, index=False)

**Analisa o retorno das revisões de cada Work Item retornado anteriormente, e cria um novo arquivo Excel para conter apenas as alterações.**

In [12]:
def analyze_changes(df):
    df.sort_values(by=['Type', 'ID', 'REV ID'], inplace=True)
    work_item_changes_dfs = {}

    # Agrupa por 'Type' e 'ID', que são as nossas chaves de identificação de cada Work Item #
    grouped = df.groupby(['Type', 'ID'])

    for (type_id, id), group in grouped:
        if len(group) < 2:
            continue  # Ignora se houver apenas uma revisão, já que não houve modificações #

        previous_row = None
        for index, current_row in group.iterrows():
            if previous_row is not None:
                changes = {'Type': type_id, 'ID': id}  # Inicializa com Type e ID, para mantermos estas informações de identificação #
                for column in group.columns:
                    if column not in ['Type', 'ID', 'REV ID'] and current_row[column] != previous_row[column]:
                        changes[column] = f"{previous_row[column]} to {current_row[column]}"

                if len(changes) > 2:  # Verifica se existem alterações além de Type e ID que inicializamos anteriormente #
                    changes['From REV ID'] = previous_row['REV ID']
                    changes['To REV ID'] = current_row['REV ID']
                    changes['Detail'] = f"From REV {previous_row['REV ID']} to {current_row['REV ID']}"
                    unique_key = f"{type_id}-{id}"

                    if unique_key not in work_item_changes_dfs:
                        work_item_changes_dfs[unique_key] = []

                    work_item_changes_dfs[unique_key].append(changes)

            previous_row = current_row

    return work_item_changes_dfs

work_item_changes_dfs = analyze_changes(combined_df) # combined_df é nosso dataframe gerado anteriormente com as revisões #

# Salva as mudanças em um novo arquivo excel #
output_changes_filename = 'work_item_changes.xlsx'
with pd.ExcelWriter(output_changes_filename) as writer:
    for unique_key, changes_list in work_item_changes_dfs.items():
        if changes_list:  # Verifica se existe alguma alteração registrada #
            df = pd.DataFrame(changes_list)
            df.to_excel(writer, sheet_name=unique_key[:31], index=False)  # Limita o nome da sheet a 31 caracteres #

